In [1]:
struct Boundary
    boundtype :: Symbol
    vertice_start :: Int64
    vertice_end :: Int64
    node_num :: Int64
end

struct MeshDef
    blocks :: Vector{Dict{Symbol, Any}}
    bounds :: Vector{Boundary}
    vertices :: Vector{Vector{Float64}}
end

In [2]:
function defineCartesian(xmin::Float64, ymin::Float64, xmax::Float64, ymax::Float64, nx::Int64, ny::Int64)
    # Create vertices
    vertices = [[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]]
    
    # Create Bounds
    bounds = Vector{Boundary}(undef, 4)
    for i in 1:4
        n = i in (1,3) ? nx : ny
        bounds[i] = Boundary(:equidistant, i, i%4+1, n)
    end

    blocks = [Dict(:type=>:cartesian, :bounds=>[1,2,3,4])]
        
    return MeshDef(blocks, bounds, vertices)
end

defineCartesian (generic function with 1 method)

In [166]:
mutable struct Mesh
    nodes :: Vector{Vector{Float64}}
    elements :: Vector{Vector{Int64}}
end

function emptyMesh()
    Mesh(Vector{Float64}[], Vector{Int64}[])
end

function generateMesh(meshdefinition::MeshDef)
    
    # disable garbage collection during mesh creation
    GC.enable(false)
    
    # create an empty Mesh object
    mesh = emptyMesh()
    
    # create Dict for Node mapping, mapping of node_ids to boundary respectiveley vertice ids
    nodeMapping = Dict(:vertice => Dict{Int64, Int64}(), :boundary => Dict{Int64, Vector{Int64}}())
    
    # create nodes on bounds
    for (boundary_id, boundary) in enumerate(meshdefinition.bounds)
        
        skipFirst = false
        if verthasNode(nodeMapping, boundary.vertice_start)
            skipFirst = true
        end
        
        skipLast = false
        if verthasNode(nodeMapping, boundary.vertice_end)
            skipLast = true
        end
        
        node_ids = appendBoundaryNodes!(mesh, boundary, meshdefinition.vertices, skipFirst, skipLast)
        
        nodeMapping[:boundary][boundary_id] = node_ids
        
        # add created nodes to nodeMapping
        if !skipFirst
            nodeMapping[:vertice][boundary.vertice_start] = node_ids[1]
        else
            insert!(nodeMapping[:boundary][boundary_id], 1, nodeMapping[:vertice][boundary.vertice_start])
        end
        if !skipLast
            nodeMapping[:vertice][boundary.vertice_end] = node_ids[end]
        else
            push!(nodeMapping[:boundary][boundary_id], nodeMapping[:vertice][boundary.vertice_end])
        end
    end
        
    # generate blocks
    for (block_id, block) in enumerate(meshdefinition.blocks)
                
        block_bounds = [meshdef.bounds[i] for i in block[:bounds]]
                
        addBlock!(mesh, block, block_bounds, nodeMapping)
    end
    
    GC.enable(true)
            
    return mesh
end

function verthasNode(nodeMapping, vert_id)
    return haskey(nodeMapping[:vertice], vert_id)
end

verthasNode (generic function with 1 method)

In [159]:
function addBlock!(mesh::Mesh, block, bounds, nodeMapping)
    if !(block[:type] == :cartesian)
        throw(DomainError())
    end
    
    # extract blocks boundaries
    boundary_ids = block[:bounds]
    boundaries = bounds[boundary_ids]
    
    # number of nodes in directions 1 (bound1, -bound3)
    #   and 2 (bound2, -bound4)
    n1 = boundaries[1].node_num
    n2 = boundaries[2].node_num
    # TODO: check other bounds, throw Exception
    
    # get boundary nodes for first boundary from nodeMapping
    node_ids = nodeMapping[:boundary][boundary_ids[1]]

    
    # coords of second bound
    coords = mesh.nodes[nodeMapping[:boundary][boundary_ids[2]]]
    
    # iterate over second dimension of block
    for i in 2:n2
        
        # store recent nodes
        last_node_ids = node_ids
        
        # generate next nodes
        if i<n2
            # calculate translation
            translation = coords[i]-coords[i-1]

            # copy nodes
            new_node_ids = translateNodes!(mesh, node_ids[2:end-1], translation, copynodes=true)
            
            node_ids = [nodeMapping[:boundary][boundary_ids[4]][end-(i-1)],
                        new_node_ids..., 
                        nodeMapping[:boundary][boundary_ids[2]][i]] 
        else
            node_ids = nodeMapping[:boundary][boundary_ids[3]][end:-1:1]
        end
            
        # all nodes of current row
        
            
        for j in 2:n1
            # create Elements
                
            el_node1 = last_node_ids[j-1]
            el_node2 = last_node_ids[j]
            el_node3 = node_ids[j]
            el_node4 = node_ids[j-1]
            
            el_nodes = [el_node1, el_node2, el_node3, el_node4]
            
            el_id = appendElement!(mesh, el_nodes)
        end
    end
end

function translateNodes!(mesh::Mesh, node_ids::Vector{Int64}, translation::Vector{Float64}; copynodes::Bool=true)
    if !copynodes
        throw(DomainError())
    end
    
    coords = []
        
    for node_id in node_ids
        push!(coords, mesh.nodes[node_id]+translation)
    end
            
    node_ids_new = appendNodes!(mesh, coords)
    
    return node_ids_new
end

translateNodes! (generic function with 1 method)

In [124]:
function appendNodes!(mesh::Mesh, nodecoords)
    start_id = size(mesh.nodes, 1)+1
    length = size(nodecoords, 1)
    
    append!(mesh.nodes, nodecoords)
    
    return collect(start_id:1:start_id+length-1)
end

function appendElement!(mesh::Mesh, node_ids)
    push!(mesh.elements, node_ids)
end

appendElement! (generic function with 1 method)

In [125]:
function appendBoundaryNodes!(mesh::Mesh, boundary::Boundary, vertices, skipFirst::Bool, skipLast::Bool)
    if boundary.boundtype != :equidistant
        throw(DomainError())
    end
    
    # get coordinates of start and end point
    coords1 = vertices[boundary.vertice_start]
    coords2 = vertices[boundary.vertice_end]

    # linear interpolation to get all coordinates    
    Δcoords = coords2-coords1
    linspace = collect(range(0, length=boundary.node_num, stop=1 ))
    nodes = [coords1 + Δcoords * ω for ω in linspace]
    
    #
    id1 = skipFirst ? 2 : 1
    id2 = skipLast ? 1 : 0
    
    nodeids = appendNodes!(mesh, nodes[id1:end-id2])
    
    return nodeids
end

appendBoundaryNodes! (generic function with 1 method)

In [98]:
using Formatting

function writeAbq(filename::String, mesh::Mesh)
    # open file
    open(filename, "w") do afile
        # node header
        write(afile, "*Node\n")
        
        # node coordinates
        for (nodeid, nodecoords) in enumerate(mesh.nodes)
            write(afile, format("{}, {}, {}\n", nodeid, nodecoords...))
        end
        
        # element header
        write(afile, "*Element, type=CPS4R\n")
        
        # elements
        for (elid, element) in enumerate(mesh.elements)
            write(afile, format("{}, {}, {}, {}, {}\n", elid, element...))
        end
    end
end

writeAbq (generic function with 1 method)

In [152]:
mesh.nodes

1500-element Array{Array{Float64,1},1}:
 [0.0, 0.0]          
 [0.0204082, 0.0]    
 [0.0408163, 0.0]    
 [0.0612245, 0.0]    
 [0.0816327, 0.0]    
 [0.102041, 0.0]     
 [0.122449, 0.0]     
 [0.142857, 0.0]     
 [0.163265, 0.0]     
 [0.183673, 0.0]     
 [0.204082, 0.0]     
 [0.22449, 0.0]      
 [0.244898, 0.0]     
 ⋮                   
 [0.755102, 0.965517]
 [0.77551, 0.965517] 
 [0.795918, 0.965517]
 [0.816327, 0.965517]
 [0.836735, 0.965517]
 [0.857143, 0.965517]
 [0.877551, 0.965517]
 [0.897959, 0.965517]
 [0.918367, 0.965517]
 [0.938776, 0.965517]
 [0.959184, 0.965517]
 [0.979592, 0.965517]

In [165]:
GC.enable(false)

Base.GC

In [175]:
meshdef = defineCartesian(0.0, 0.0, 1.0, 10.0, 1000, 500)

@time mesh = generateMesh(meshdef);

  0.380032 seconds (5.48 M allocations: 209.554 MiB, 24.06% gc time)


In [170]:
writeAbq("test.inp", mesh)

In [171]:
varinfo()

| name                 |       size | summary                      |
|:-------------------- | ----------:|:---------------------------- |
| Base                 |            | Module                       |
| Boundary             |  212 bytes | DataType                     |
| Core                 |            | Module                       |
| Main                 |            | Module                       |
| Mesh                 |  196 bytes | DataType                     |
| MeshDef              |  204 bytes | DataType                     |
| a                    |   72 bytes | 4-element Array{Int64,1}     |
| addBlock!            |    0 bytes | typeof(addBlock!)            |
| addElement!          |    0 bytes | typeof(addElement!)          |
| appendBoundaryNodes! |    0 bytes | typeof(appendBoundaryNodes!) |
| appendElement!       |    0 bytes | typeof(appendElement!)       |
| appendNodes!         |    0 bytes | typeof(appendNodes!)         |
| defineCartesian      |    0 bytes | typeof(defineCartesian)      |
| emptyMesh            |    0 bytes | typeof(emptyMesh)            |
| generateMesh         |    0 bytes | typeof(generateMesh)         |
| mesh                 | 20.538 MiB | Mesh                         |
| meshdef              |  1.070 KiB | MeshDef                      |
| startupfile          |   50 bytes | String                       |
| translateNodes!      |    0 bytes | typeof(translateNodes!)      |
| verthasNode          |    0 bytes | typeof(verthasNode)          |
| writeAbq             |    0 bytes | typeof(writeAbq)             |
